In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import os
import cv2
import numpy as np
import pandas as pd

# Dog Breed Identification using Transfer Learning

Objective is to fine tune a pretrained object classification model such as resnet/inception/vgg16 trained on large datasets and use their feature vector to build a fine-tuned model for specific purpose of dog-breed identification. 
Detailed description of problem staetment can be found on official page for competetion: https://www.kaggle.com/c/dog-breed-identification. 

### Description of training data
10222 images of Dogs, 120 classes.

### Procedure

1. Import a pre-trained network : Resnet V2 (101 layers) is imported using tensorflow hub.
2. Pre-process and augment input data for Resnet V2 ( 224x224x3 )
3. Extract feature embedding for each image using Resnet and build a ML model upon that for breed identification task.

In [2]:
tf.logging.set_verbosity('INFO')

# set trainable true/False -> fine tune entire model for input data
module = hub.Module("https://tfhub.dev/google/imagenet/resnet_v2_101/feature_vector/1", trainable=True);
height, width = hub.get_expected_image_size(module);
shape = (height, width)

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Initialize variable module/resnet_v2_101/block1/unit_1/bottleneck_v2/conv1/BatchNorm/beta:0 from checkpoint b'/tmp/tfhub_modules/e0c607f95a3d67bc8928a5c20d09d1915322cfcb/variables/variables' with resnet_v2_101/block1/unit_1/bottleneck_v2/conv1/BatchNorm/beta
INFO:tensorflow:Initialize variable module/resnet_v2_101/block1/unit_1/bottleneck_v2/conv1/BatchNorm/gamma:0 from checkpoint b'/tmp/tfhub_modules/e0c607f95a3d67bc8928a5c20d09d1915322cfcb/variables/variables' with resnet_v2_101/block1/unit_1/bottleneck_v2/conv1/BatchNorm/gamma
INFO:tensorflow:Initialize variable module/resnet_v2_101/block1/unit_1/bottleneck_v2/conv1/BatchNorm/moving_mean:0 from checkpoint b'/tmp/tfhub_modules/e0c607f95a3d67bc8928a5c20d09d1915322cfcb/variables/variables' with resnet_v2_101/block1/unit_1/bottleneck_v2/conv1/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/resnet_v2_101/block1/unit_1/bottleneck_v2/conv1/BatchNorm

INFO:tensorflow:Initialize variable module/resnet_v2_101/block1/unit_2/bottleneck_v2/preact/beta:0 from checkpoint b'/tmp/tfhub_modules/e0c607f95a3d67bc8928a5c20d09d1915322cfcb/variables/variables' with resnet_v2_101/block1/unit_2/bottleneck_v2/preact/beta
INFO:tensorflow:Initialize variable module/resnet_v2_101/block1/unit_2/bottleneck_v2/preact/gamma:0 from checkpoint b'/tmp/tfhub_modules/e0c607f95a3d67bc8928a5c20d09d1915322cfcb/variables/variables' with resnet_v2_101/block1/unit_2/bottleneck_v2/preact/gamma
INFO:tensorflow:Initialize variable module/resnet_v2_101/block1/unit_2/bottleneck_v2/preact/moving_mean:0 from checkpoint b'/tmp/tfhub_modules/e0c607f95a3d67bc8928a5c20d09d1915322cfcb/variables/variables' with resnet_v2_101/block1/unit_2/bottleneck_v2/preact/moving_mean
INFO:tensorflow:Initialize variable module/resnet_v2_101/block1/unit_2/bottleneck_v2/preact/moving_variance:0 from checkpoint b'/tmp/tfhub_modules/e0c607f95a3d67bc8928a5c20d09d1915322cfcb/variables/variables' with

INFO:tensorflow:Initialize variable module/resnet_v2_101/block2/unit_1/bottleneck_v2/conv3/biases:0 from checkpoint b'/tmp/tfhub_modules/e0c607f95a3d67bc8928a5c20d09d1915322cfcb/variables/variables' with resnet_v2_101/block2/unit_1/bottleneck_v2/conv3/biases
INFO:tensorflow:Initialize variable module/resnet_v2_101/block2/unit_1/bottleneck_v2/conv3/weights:0 from checkpoint b'/tmp/tfhub_modules/e0c607f95a3d67bc8928a5c20d09d1915322cfcb/variables/variables' with resnet_v2_101/block2/unit_1/bottleneck_v2/conv3/weights
INFO:tensorflow:Initialize variable module/resnet_v2_101/block2/unit_1/bottleneck_v2/preact/beta:0 from checkpoint b'/tmp/tfhub_modules/e0c607f95a3d67bc8928a5c20d09d1915322cfcb/variables/variables' with resnet_v2_101/block2/unit_1/bottleneck_v2/preact/beta
INFO:tensorflow:Initialize variable module/resnet_v2_101/block2/unit_1/bottleneck_v2/preact/gamma:0 from checkpoint b'/tmp/tfhub_modules/e0c607f95a3d67bc8928a5c20d09d1915322cfcb/variables/variables' with resnet_v2_101/block

INFO:tensorflow:Initialize variable module/resnet_v2_101/block2/unit_3/bottleneck_v2/conv2/BatchNorm/moving_mean:0 from checkpoint b'/tmp/tfhub_modules/e0c607f95a3d67bc8928a5c20d09d1915322cfcb/variables/variables' with resnet_v2_101/block2/unit_3/bottleneck_v2/conv2/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/resnet_v2_101/block2/unit_3/bottleneck_v2/conv2/BatchNorm/moving_variance:0 from checkpoint b'/tmp/tfhub_modules/e0c607f95a3d67bc8928a5c20d09d1915322cfcb/variables/variables' with resnet_v2_101/block2/unit_3/bottleneck_v2/conv2/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable module/resnet_v2_101/block2/unit_3/bottleneck_v2/conv2/weights:0 from checkpoint b'/tmp/tfhub_modules/e0c607f95a3d67bc8928a5c20d09d1915322cfcb/variables/variables' with resnet_v2_101/block2/unit_3/bottleneck_v2/conv2/weights
INFO:tensorflow:Initialize variable module/resnet_v2_101/block2/unit_3/bottleneck_v2/conv3/biases:0 from checkpoint b'/tmp/tfhub_modules/e0c607f95a3d67bc

INFO:tensorflow:Initialize variable module/resnet_v2_101/block3/unit_1/bottleneck_v2/conv2/BatchNorm/beta:0 from checkpoint b'/tmp/tfhub_modules/e0c607f95a3d67bc8928a5c20d09d1915322cfcb/variables/variables' with resnet_v2_101/block3/unit_1/bottleneck_v2/conv2/BatchNorm/beta
INFO:tensorflow:Initialize variable module/resnet_v2_101/block3/unit_1/bottleneck_v2/conv2/BatchNorm/gamma:0 from checkpoint b'/tmp/tfhub_modules/e0c607f95a3d67bc8928a5c20d09d1915322cfcb/variables/variables' with resnet_v2_101/block3/unit_1/bottleneck_v2/conv2/BatchNorm/gamma
INFO:tensorflow:Initialize variable module/resnet_v2_101/block3/unit_1/bottleneck_v2/conv2/BatchNorm/moving_mean:0 from checkpoint b'/tmp/tfhub_modules/e0c607f95a3d67bc8928a5c20d09d1915322cfcb/variables/variables' with resnet_v2_101/block3/unit_1/bottleneck_v2/conv2/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/resnet_v2_101/block3/unit_1/bottleneck_v2/conv2/BatchNorm/moving_variance:0 from checkpoint b'/tmp/tfhub_modules/e0c

...

INFO:tensorflow:Initialize variable module/resnet_v2_101/block3/unit_4/bottleneck_v2/conv1/BatchNorm/beta:0 from checkpoint b'/tmp/tfhub_modules/e0c607f95a3d67bc8928a5c20d09d1915322cfcb/variables/variables' with resnet_v2_101/block3/unit_4/bottleneck_v2/conv1/BatchNorm/beta
INFO:tensorflow:Initialize variable module/resnet_v2_101/block3/unit_4/bottleneck_v2/conv1/BatchNorm/gamma:0 from checkpoint b'/tmp/tfhub_modules/e0c607f95a3d67bc8928a5c20d09d1915322cfcb/variables/variables' with resnet_v2_101/block3/unit_4/bottleneck_v2/conv1/BatchNorm/gamma
INFO:tensorflow:Initialize variable module/resnet_v2_101/block3/unit_4/bottleneck_v2/conv1/BatchNorm/moving_mean:0 from checkpoint b'/tmp/tfhub_modules/e0c607f95a3d67bc8928a5c20d09d1915322cfcb/variables/variables' with resnet_v2_101/block3/unit_4/bottleneck_v2/conv1/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/resnet_v2_101/block3/unit_4/bottleneck_v2/conv1/BatchNorm/moving_variance:0 from checkpoint b'/tmp/tfhub_modules/e0c

INFO:tensorflow:Initialize variable module/resnet_v2_101/block3/unit_5/bottleneck_v2/preact/moving_mean:0 from checkpoint b'/tmp/tfhub_modules/e0c607f95a3d67bc8928a5c20d09d1915322cfcb/variables/variables' with resnet_v2_101/block3/unit_5/bottleneck_v2/preact/moving_mean
INFO:tensorflow:Initialize variable module/resnet_v2_101/block3/unit_5/bottleneck_v2/preact/moving_variance:0 from checkpoint b'/tmp/tfhub_modules/e0c607f95a3d67bc8928a5c20d09d1915322cfcb/variables/variables' with resnet_v2_101/block3/unit_5/bottleneck_v2/preact/moving_variance
INFO:tensorflow:Initialize variable module/resnet_v2_101/block3/unit_6/bottleneck_v2/conv1/BatchNorm/beta:0 from checkpoint b'/tmp/tfhub_modules/e0c607f95a3d67bc8928a5c20d09d1915322cfcb/variables/variables' with resnet_v2_101/block3/unit_6/bottleneck_v2/conv1/BatchNorm/beta
INFO:tensorflow:Initialize variable module/resnet_v2_101/block3/unit_6/bottleneck_v2/conv1/BatchNorm/gamma:0 from checkpoint b'/tmp/tfhub_modules/e0c607f95a3d67bc8928a5c20d09d

INFO:tensorflow:Initialize variable module/resnet_v2_101/block3/unit_7/bottleneck_v2/preact/beta:0 from checkpoint b'/tmp/tfhub_modules/e0c607f95a3d67bc8928a5c20d09d1915322cfcb/variables/variables' with resnet_v2_101/block3/unit_7/bottleneck_v2/preact/beta
INFO:tensorflow:Initialize variable module/resnet_v2_101/block3/unit_7/bottleneck_v2/preact/gamma:0 from checkpoint b'/tmp/tfhub_modules/e0c607f95a3d67bc8928a5c20d09d1915322cfcb/variables/variables' with resnet_v2_101/block3/unit_7/bottleneck_v2/preact/gamma
INFO:tensorflow:Initialize variable module/resnet_v2_101/block3/unit_7/bottleneck_v2/preact/moving_mean:0 from checkpoint b'/tmp/tfhub_modules/e0c607f95a3d67bc8928a5c20d09d1915322cfcb/variables/variables' with resnet_v2_101/block3/unit_7/bottleneck_v2/preact/moving_mean
INFO:tensorflow:Initialize variable module/resnet_v2_101/block3/unit_7/bottleneck_v2/preact/moving_variance:0 from checkpoint b'/tmp/tfhub_modules/e0c607f95a3d67bc8928a5c20d09d1915322cfcb/variables/variables' with

INFO:tensorflow:Initialize variable module/resnet_v2_101/block3/unit_9/bottleneck_v2/conv3/biases:0 from checkpoint b'/tmp/tfhub_modules/e0c607f95a3d67bc8928a5c20d09d1915322cfcb/variables/variables' with resnet_v2_101/block3/unit_9/bottleneck_v2/conv3/biases
INFO:tensorflow:Initialize variable module/resnet_v2_101/block3/unit_9/bottleneck_v2/conv3/weights:0 from checkpoint b'/tmp/tfhub_modules/e0c607f95a3d67bc8928a5c20d09d1915322cfcb/variables/variables' with resnet_v2_101/block3/unit_9/bottleneck_v2/conv3/weights
INFO:tensorflow:Initialize variable module/resnet_v2_101/block3/unit_9/bottleneck_v2/preact/beta:0 from checkpoint b'/tmp/tfhub_modules/e0c607f95a3d67bc8928a5c20d09d1915322cfcb/variables/variables' with resnet_v2_101/block3/unit_9/bottleneck_v2/preact/beta
INFO:tensorflow:Initialize variable module/resnet_v2_101/block3/unit_9/bottleneck_v2/preact/gamma:0 from checkpoint b'/tmp/tfhub_modules/e0c607f95a3d67bc8928a5c20d09d1915322cfcb/variables/variables' with resnet_v2_101/block

INFO:tensorflow:Initialize variable module/resnet_v2_101/block4/unit_2/bottleneck_v2/conv2/BatchNorm/moving_mean:0 from checkpoint b'/tmp/tfhub_modules/e0c607f95a3d67bc8928a5c20d09d1915322cfcb/variables/variables' with resnet_v2_101/block4/unit_2/bottleneck_v2/conv2/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/resnet_v2_101/block4/unit_2/bottleneck_v2/conv2/BatchNorm/moving_variance:0 from checkpoint b'/tmp/tfhub_modules/e0c607f95a3d67bc8928a5c20d09d1915322cfcb/variables/variables' with resnet_v2_101/block4/unit_2/bottleneck_v2/conv2/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable module/resnet_v2_101/block4/unit_2/bottleneck_v2/conv2/weights:0 from checkpoint b'/tmp/tfhub_modules/e0c607f95a3d67bc8928a5c20d09d1915322cfcb/variables/variables' with resnet_v2_101/block4/unit_2/bottleneck_v2/conv2/weights
INFO:tensorflow:Initialize variable module/resnet_v2_101/block4/unit_2/bottleneck_v2/conv3/biases:0 from checkpoint b'/tmp/tfhub_modules/e0c607f95a3d67bc

In [3]:
# tf.global_variables()
# module.variable_map
LOG_DIR = '.graph'

In [4]:
# resize images in train folder if not in shape
def resize(shape=shape):
    
    originalFolder = 'train'
    resizedFolder  = 'resTrain'
    
    if not os.path.exists(resizedFolder):
        os.makedirs(resizedFolder)
        counter = 0
        # Iterate through resizing and saving
        for img in os.listdir('train'):
            counter = counter + 1
            pic = cv2.imread("train/" + img, cv2.IMREAD_UNCHANGED)
            pic = cv2.resize(pic, shape)
            cv2.imwrite(resizedFolder + '/' + img, pic)
            print("resized image : " + str(shape) + str(counter) + " : "+img + " : " );
    else:
        print("Rezied image already exists in folder:" + resizedFolder);
        return

# resize images to resnet accepted shape
resize(shape)

Rezied image already exists in folder:resTrain


In [6]:
label = pd.read_csv("labels.csv", delimiter=',', index_col=0) # 120 labels
labelOneHot = pd.get_dummies(label.breed) # 120 labels

global curr_batch  # flag to track current batch being fed in each epoch
curr_batch = 0     # first initialization, re-initialize with caution

'''
input batch-size, default = 100
uses global variable curr_batch to remember current batch location
returns tuple (flag for next batch, list of files, list of labels, 4-d array->(batch, h, w, c))
'''
def next_batch(batch_size=100, random=False):
    next_batch = True
    if random:
        label_list = labelOneHot.sample(n=batch_size, replace=False)
        batch_list = label_list.index
    else:
        global curr_batch
        start = curr_batch*batch_size
        end   = (curr_batch+1)*batch_size
        # print("feeding Batch : " + str(curr_batch))
        # Check is it's last batch
        if ((curr_batch+1)*batch_size>len(label)):
            end = len(label)-1
            next_batch = False
        batch_list = label.index[start:end]
        label_list = labelOneHot[start:end]
    
    batch = []
    for img in batch_list:
        pic = cv2.imread("resTrain/"+img+".jpg", cv2.IMREAD_UNCHANGED)
        batch.append(pic)
    batch = np.stack(batch, axis=0)
    if not(random):
        curr_batch = curr_batch + 1
    
    return next_batch, batch_list, label_list, batch


In [7]:
# Define retrain layer
with tf.name_scope("retrain_model"):
    # Input placeholders
    images = tf.placeholder(dtype=tf.float32, shape=(None, height, width, 3), name="input_images")
    y      = tf.placeholder(dtype=tf.int32, shape=(None, 120), name="labels")

    # bottleneck
    features = module(images)  # Features with shape [batch_size, 2048], last batch may differ in size
    # retraining model 
    dense1 = tf.layers.dense(inputs=features, units=1024, activation=tf.nn.relu, name="dense_1")
    logits = tf.layers.dense(inputs=dense1, units=120, name="dense_2")
    predictions = {
        "classes": tf.argmax(input=logits, axis=1),
        "probabilities": tf.nn.softmax(logits, name="softmax")
    }
    xent = tf.reduce_mean(tf.losses.softmax_cross_entropy(onehot_labels=y, logits=logits))

    train_step = tf.train.GradientDescentOptimizer(learning_rate=0.01) \
                    .minimize(loss=xent, global_step=tf.train.get_global_step())

    eval_metric = {
        "accuracy": tf.metrics.accuracy(labels=y, predictions=predictions["probabilities"])}

    tf.summary.scalar('cross_entropy', xent)
    tf.summary.scalar('accuracy', eval_metric["accuracy"][0])

    # Set up logging for predictions
    tensors_to_log = {"probabilities":predictions["probabilities"], 
                      "accuracy": eval_metric["accuracy"], 
                      "train_cost": xent}
    logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=1)

    summ = tf.summary.merge_all()

In [12]:
batchNext = True
curr_batch = 0
counter = 0

with tf.Session() as sess:
    init_g = tf.global_variables_initializer()
    init_l = tf.local_variables_initializer()
    writer = tf.summary.FileWriter(LOG_DIR, sess.graph)
    saver  = tf.train.Saver()
    sess.run(init_g)
    sess.run(init_l)
    
    for epoch in range(10):
        curr_batch = 0
        while(batchNext):
            batchNext, _, batch_label, image = next_batch(25)  # A batch of images with shape [batch_size, height, width, 3].
            _, _, batch_label_cv, image_cv = next_batch(25, True) # randomly sample images for cross-validation [batch_size, height, width, 3].

            [train_accuracy, s] = sess.run([eval_metric["accuracy"], summ], feed_dict={images: image, y: batch_label})
            [train_accuracy_cv, s_cv] = sess.run([eval_metric["accuracy"], summ], feed_dict={images: image_cv, y: batch_label_cv})
            writer.add_summary(s, counter)
            writer.add_summary(s_cv, counter)
            print("Epoch: %2d, Iteration: %3d, training accuracy : %6.5f , cv_accuracy: %6.5f"%(epoch, counter, train_accuracy[0], train_accuracy_cv[0]))
            if counter % 20 == 0:
        #         sess.run(assignment, feed_dict={x: , y: mnist.test.labels[:1024]})
                saver.save(sess, os.path.join(LOG_DIR, "kaggle-dog-identification.ckpt"), counter)
            sess.run(train_step, feed_dict={images: image, y: batch_label})
            counter = counter+1      


Epoch:  0, Iteration:   0, training accuracy : 0.00000 , cv_accuracy: 0.98333
Epoch:  0, Iteration:   1, training accuracy : 0.98333 , cv_accuracy: 0.98333
Epoch:  0, Iteration:   2, training accuracy : 0.98333 , cv_accuracy: 0.98360
Epoch:  0, Iteration:   3, training accuracy : 0.98356 , cv_accuracy: 0.98352
Epoch:  0, Iteration:   4, training accuracy : 0.98358 , cv_accuracy: 0.98356


KeyboardInterrupt: 

In [ ]:
pts1 = np.float32([[50,50],[200,50],[50,200]])
pts2 = np.float32([[10,100],[200,50],[100,250]])


In [ ]:
pts1.shape